In [8]:
import os
import cv2
import json
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from matplotlib.pyplot import imshow
%matplotlib inline

In [ ]:
model_path = input("Insert path to the Model: \t")
crop_top = int(input("Insert model crop top value: \t"))
tub_path = input("Insert path to the tub of the track: \t")
tub_path = tub_path + '/' if tub_path[-1] != '/' else tub_path

while not os.path.isdir(tub_path) or not os.path.isfile(model_path):
    print("One of the paths was incorrect")

    model_path = input("Insert path to the Model: \t")
    tub_path = input("Insert path to the tub of the track: \t")
    tub_path = tub_path + '/' if tub_path[-1] != '/' else tub_path


In [39]:
# Alternatively, put paths and crop manually
model_path = "/Users/brianpulfer/mycar/thesis/models/sim/tests/cropping/crop120.h5"
tub_path = "/Users/brianpulfer/mycar/thesis/data/tub320x240"
tub_path = tub_path + '/' if tub_path[-1] != '/' else tub_path
crop_top = 120

In [40]:
# Loading model
model = keras.models.load_model(model_path)

# Getting model input size
MODEL_INPUT_SIZE = model.layers[0]._batch_input_shape[1:3]
MODEL_INPUT_SIZE = (MODEL_INPUT_SIZE[1], MODEL_INPUT_SIZE[0])

# Getting dataset image size
images_names = [name for name in sorted(os.listdir(tub_path)) if 'jpg' in name.lower()]
first_image = Image.open(tub_path + images_names[0])

w, h = (first_image.size[0], first_image.size[1])
print("Detected images size: ", first_image.size)
print("Image size after top crop: ", (w, h-crop_top))
print("Detected model input size: ", MODEL_INPUT_SIZE)

if MODEL_INPUT_SIZE != (w, h-crop_top):
    print("WARNING: Input sizes do not match! Reshaping might yield bad predictions")
else:
    print("Sizes seem to match\n")
    
model.summary()

Detected images size:  (320, 240)
Image size after top crop:  (320, 120)
Detected model input size:  (320, 120)
Sizes seem to match

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_in (InputLayer)             (None, 120, 320, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 58, 158, 24)  1824        img_in[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 58, 158, 24)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 27, 77, 32)   19232       dropout[0]

In [41]:
def get_dataset(tub_path, w, h, crop_top, dave2=False):
    # Collecting dataset in a tensor
    n = len([i for i in os.listdir(tub_path) if 'jpg' in i.lower()])
    X, Y = [], []
    for image_name in tqdm(images_names):
        if 'jpg' not in image_name.lower():
            continue

        # Defining paths
        image_path = os.path.join(tub_path, image_name)
        record_path = os.path.join(tub_path, "record_" + image_name.split("_")[0] + ".json")

        # Loading image and relative record
        image = Image.open(image_path)
        resized = image.resize((w, h))
        normalized = np.array(resized) / 255.0
        cropped = normalized[crop_top:, ...]

        if dave2:
            cropped = cv2.cvtColor((cropped*255).astype('uint8'), cv2.COLOR_RGB2YUV)
            cropped = cropped.astype('float') / 255

        record = json.load(open(record_path))

        # Loading labels
        angle = record['user/angle']
        throttle = record['user/throttle']

        # Adding data in the tensor
        X.append(np.array(cropped).astype(np.float32))
        Y.append(np.array([angle, throttle]))

    return np.array(X), np.array(Y)

X, Y = get_dataset(tub_path, w, h, crop_top, dave2=False)

100%|██████████| 3657/3657 [00:05<00:00, 641.59it/s]


In [42]:
# Getting the predictions
Y_hat = model.predict(X)
angle_predictions = Y_hat[0]
throttle_predictions = Y_hat[1]

In [43]:
# Evaluating model offline
mse_steer, mse_throttle = 0, 0
mae_steer, mae_throttle = 0, 0
for i in range(len(X)):
    s, t = Y[i][0], Y[i][1]
    s_hat, t_hat = angle_predictions[i][0], throttle_predictions[i][0]

    mse_steer += (s_hat - s)**2
    mse_throttle += (t_hat - t)**2

    mae_steer += abs(s_hat - s)
    mae_throttle += abs(t_hat -t)

# Getting average
mse_steer /= len(X)
mse_throttle /= len(X)
mae_steer /= len(X)
mae_throttle /= len(X)

# Getting total loss
loss = mse_steer + mse_throttle

# Printing
print("Steering MSE: {}\t\tThrottle MSE: {}".format(mse_steer, mse_throttle))
print("Steering MAE: {}\t\tThrottle MAE: {}".format(mae_steer, mae_throttle))
print("Total LOSS: {}".format(loss))

Steering MSE: 0.08794156657014457		Throttle MSE: 0.0032620402743986842
Steering MAE: 0.20881679831433678		Throttle MAE: 0.03461342960450801
Total LOSS: 0.09120360684454325
